In [1]:
import datetime
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import warnings

from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


FEATS_EXCLUDED = ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0']
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# rmse
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
    
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]

    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

# reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
    


# LightGBM GBDT with KFold or Stratified KFold
def kfold_lightgbm(train_df, test_df, num_folds, stratified = False, debug= False):
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=326)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=326)

    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]

    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['outliers'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['target'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['target'].iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params ={
                'task': 'train',
                'boosting': 'goss',
                'objective': 'regression',
                'metric': 'rmse',
                'learning_rate': 0.01,
                'subsample': 0.9855232997390695,
                'max_depth': 7,
                'top_rate': 0.9064148448434349,
                'num_leaves': 63,
                'min_child_weight': 41.9612869171337,
                'other_rate': 0.0721768246018207,
                'reg_alpha': 9.677537745007898,
                'colsample_bytree': 0.5665320670155495,
                'min_split_gain': 9.820197773625843,
                'reg_lambda': 8.2532317400459,
                'min_data_in_leaf': 21,
                'verbose': -1,
                'seed':int(2**n_fold),
                'bagging_seed':int(2**n_fold),
                'drop_seed':int(2**n_fold)
                }

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=100
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test_df[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()

    # display importances
    display_importances(feature_importance_df)

    if not debug:
        # save submission file
        test_df.loc[:,'target'] = sub_preds
        test_df = test_df.reset_index()
        test_df[['card_id', 'target']].to_csv(submission_file_name, index=False)


In [2]:
%%time
train_df = reduce_mem_usage(pd.read_csv('train_clean.csv'))
test_df = reduce_mem_usage(pd.read_csv('test_clean.csv'))

Memory usage after optimization is: 94.16 MB
Decreased by 69.9%


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in less
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in less


Memory usage after optimization is: 57.65 MB
Decreased by 69.9%
Wall time: 33.7 s


In [3]:
import xgboost as xgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def objective(trial):
    
        data = train_df[feats]
        target = train_df['target']
        train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
        dtrain = xgb.DMatrix(train_x, label=train_y)
        dtest = xgb.DMatrix(test_x, label=test_y)
        

        param = {'objective': 'reg:linear',
                  'eval_metric': 'rmse',
                  'verbosity': -1,
                  "learning_rate": trial.suggest_uniform('learning_rate', 0.001, 1),
                  
                  'silent': 1,
                  'booster': trial.suggest_categorical('booster', ['gbtree','dart']),
                  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
                  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
                  
                  #'gpu_id': 0,
                  #'tree_method': 'gpu_hist',
                  #'max_bin': 16,
                  #'updater' : 'grow_gpu_hist',

                
                  'seed': 326,
                  'boosting_type': trial.suggest_categorical('boosting', ['gbdt',  'goss']),
                  'num_leaves': trial.suggest_int('num_leaves', 16, 64),
                  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.001, 1),
                  'subsample': trial.suggest_uniform('subsample', 0.001, 1),
                  'max_depth': trial.suggest_int('max_depth', 5, 20),
                  #'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 10),
                  'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 10),
                  #'min_split_gain': trial.suggest_uniform('min_split_gain', 0, 10),
                  #'min_child_weight': trial.suggest_uniform('min_child_weight', 0, 45),
                  #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 16, 64)
                  }


        
        
        if param['booster'] == 'gbtree' or param['booster'] == 'dart':
            param['max_depth'] = trial.suggest_int('max_depth', 1, 9)
            param['eta'] = trial.suggest_loguniform('eta', 1e-8, 1.0)
            param['gamma'] = trial.suggest_loguniform('gamma', 1e-8, 1.0)
            param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
        if param['booster'] == 'dart':
            param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
            param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
            param['rate_drop'] = trial.suggest_loguniform('rate_drop', 1e-8, 1.0)
            param['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)

        gbm = xgb.train(param, dtrain)
        preds = gbm.predict(dtest)
        print(preds)
        pred_labels = np.rint(preds)
        error =  rmse(test_y,pred_labels)
        
        return error



Using TensorFlow backend.


In [4]:
with timer("split train & test"):
        '''train_df = reduce_mem_usage(pd.read_csv('train_clean.csv'))
        test_df = reduce_mem_usage(pd.read_csv('test_clean.csv'))'''
       
        feature_importance_df = pd.DataFrame()
        feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]
        
        study = optuna.create_study()
        study.optimize(objective, n_trials=100)

        print('Number of finished trials: {}'.format(len(study.trials)))

        print('Best trial:')
        trial = study.best_trial

        print('  Value: {}'.format(trial.value))

        print('  Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))
        
        hist_df = study.trials_dataframe()
        hist_df.to_csv("optuna_result_xgbm.csv")

        del df
        gc.collect()

[-0.0926857   1.3151143   0.18114877 ...  0.42111242 -4.0382833
 -1.3636883 ]


[I 2019-02-15 23:58:17,598] Finished a trial resulted in value: 4.027097702026367. Current best value is 4.027097702026367 with parameters: {'learning_rate': 0.2521224347953242, 'booster': 'dart', 'lambda': 0.0001233651023072927, 'alpha': 1.3017861984037023e-05, 'boosting': 'goss', 'num_leaves': 21, 'colsample_bytree': 0.29784878529407743, 'subsample': 0.029003905464427028, 'max_depth': 13, 'reg_lambda': 3.6257758917501635, 'eta': 0.10289744154207395, 'gamma': 0.0002816849545098529, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 3.1417711585387727e-08, 'skip_drop': 1.2844990651753047e-06}.


[ 1.5431254  -0.42633516 -0.73449147 ...  2.9881396   1.9011275
 -7.2503304 ]


[I 2019-02-16 00:03:02,825] Finished a trial resulted in value: 3.9500226974487305. Current best value is 3.9500226974487305 with parameters: {'learning_rate': 0.5115072770753109, 'booster': 'dart', 'lambda': 9.036219587627254e-08, 'alpha': 0.0032012914715939833, 'boosting': 'goss', 'num_leaves': 27, 'colsample_bytree': 0.5348230368701687, 'subsample': 0.985144462432532, 'max_depth': 19, 'reg_lambda': 5.396074182445668, 'eta': 0.19212996682810954, 'gamma': 0.00019209988132236713, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.010730596673154014, 'skip_drop': 1.898353420849061e-08}.


[-0.18087661 -2.0655363  -0.22937906 ... -0.33819646  0.8531803
 -0.29289764]


...

[I 2019-02-16 01:34:18,763] Finished a trial resulted in value: 3.690701723098755. Current best value is 3.6498684883117676 with parameters: {'learning_rate': 0.5843131085630997, 'booster': 'gbtree', 'lambda': 1.534521654591073e-07, 'alpha': 1.0239095745311145e-08, 'boosting': 'gbdt', 'num_leaves': 31, 'colsample_bytree': 0.5406862297709868, 'subsample': 0.9594952525792275, 'max_depth': 5, 'reg_lambda': 3.143121436123109, 'eta': 2.9162386740282797e-07, 'gamma': 6.015464829655147e-07, 'grow_policy': 'depthwise'}.


[-0.01237941 -0.31568915 -0.38106143 ...  0.8859701  -0.11560047
  0.09249765]


[I 2019-02-16 01:34:26,559] Finished a trial resulted in value: 3.841815233230591. Current best value is 3.6498684883117676 with parameters: {'learning_rate': 0.5843131085630997, 'booster': 'gbtree', 'lambda': 1.534521654591073e-07, 'alpha': 1.0239095745311145e-08, 'boosting': 'gbdt', 'num_leaves': 31, 'colsample_bytree': 0.5406862297709868, 'subsample': 0.9594952525792275, 'max_depth': 5, 'reg_lambda': 3.143121436123109, 'eta': 2.9162386740282797e-07, 'gamma': 6.015464829655147e-07, 'grow_policy': 'depthwise'}.


Number of finished trials: 100
Best trial:
  Value: 3.6498684883117676
  Params: 
    learning_rate: 0.5843131085630997
    booster: gbtree
    lambda: 1.534521654591073e-07
    alpha: 1.0239095745311145e-08
    boosting: gbdt
    num_leaves: 31
    colsample_bytree: 0.5406862297709868
    subsample: 0.9594952525792275
    max_depth: 5
    reg_lambda: 3.143121436123109
    eta: 2.9162386740282797e-07
    gamma: 6.015464829655147e-07
    grow_policy: depthwise


NameError: name 'df' is not defined

In [ ]:
#--------------------------------------